<a href="https://colab.research.google.com/github/Zarathos01/Machine-learning-is-fun/blob/main/Twitter%20Arabic%20Sentiment%20analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mksaad/arabic-sentiment-twitter-corpus")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/arabic-sentiment-twitter-corpus


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
!pip install emoji -q

In [ ]:
train_pos = pd.read_csv(path + "/train_Arabic_tweets_positive_20190413.tsv", sep='\t', header=None)
train_neg = pd.read_csv(path + "/train_Arabic_tweets_negative_20190413.tsv", sep='\t', header=None)
test_pos = pd.read_csv(path + "/test_Arabic_tweets_positive_20190413.tsv", sep='\t', header=None)
test_neg = pd.read_csv(path + "/test_Arabic_tweets_negative_20190413.tsv", sep='\t', header=None)

In [ ]:
train_pos.head()

,0,1
0,pos,نحن الذين يتحول كل ما نود أن نقوله إلى دعاء لل...
1,pos,وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال...
2,pos,من الخير نفسه 💛
3,pos,#زلزل_الملعب_نصرنا_بيلعب كن عالي الهمه ولا ترض...
4,pos,الشيء الوحيد الذي وصلوا فيه للعالمية هو : المس...


In [ ]:
train_neg.head()

,0,1
0,neg,اعترف ان بتس كانو شوي شوي يجيبو راسي لكن اليوم...
1,neg,توقعت اذا جات داريا بشوفهم كاملين بس لي للحين ...
2,neg,#الاهلي_الهلال اكتب توقعك لنتيجة لقاء الهلال و...
3,neg,نعمة المضادات الحيوية . تضع قطرة💧مضاد بنسلين ع...
4,neg,الدودو جايه تكمل علي 💔


In [ ]:
train_pos = train_pos.rename(columns={0: "Sentiment", 1: "comment"})
train_neg = train_neg.rename(columns={0: "Sentiment", 1: "comment"})
test_pos = test_pos.rename(columns={0: "Sentiment", 1: "comment"})
test_neg = test_neg.rename(columns={0: "Sentiment", 1: "comment"})

In [ ]:
train_data = pd.concat([train_pos, train_neg])
test_data = pd.concat([test_pos, test_neg])

In [ ]:
train_data

,Sentiment,comment
0,pos,نحن الذين يتحول كل ما نود أن نقوله إلى دعاء لل...
1,pos,وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال...
2,pos,من الخير نفسه 💛
3,pos,#زلزل_الملعب_نصرنا_بيلعب كن عالي الهمه ولا ترض...
4,pos,الشيء الوحيد الذي وصلوا فيه للعالمية هو : المس...
...,...,...
22509,neg,كيف ترى أورانوس لو كان يقع مكان القمر ؟ 💙💙 كوك...
22510,neg,احسدك على الايم 💔
22511,neg,لأول مرة ما بنكون سوا 💔
22512,neg,بقله ليش يا واطي 🤔


In [ ]:
#random the data
train_data = train_data.sample(frac=1).reset_index(drop=True)
test_data = test_data.sample(frac=1).reset_index(drop=True)

In [ ]:
train_data

,Sentiment,comment
0,pos,🚨 لزيادة متابعينك ، ضيفني + لايك + رتويت + ضيف...
1,pos,تصافحنا الحياة ..🌺 لنبتسم لليوم ..لنضحك للغد ل...
2,neg,كلما #ارغمت نفسي علي عدم #تذكرك يتصارع قلبي وي...
3,neg,خساره عاد كانت نومة الظالم عباده 😔
4,neg,يخي شهالناس السعاده :(
...,...,...
45270,neg,💘💘مالتي ماكا + فوتولايز💘💘 مميزات 💑 م💊التي ماك💊...
45271,neg,حسبي الله للحين صامل 😷
45272,neg,لغة الارقام 😣
45273,neg,داخلي صرخة الم لو اطلعها ل/ صم منهاا الكون 💔 #...


In [ ]:
train_data.shape

(45275, 2)

In [ ]:
test_data.shape

(11520, 2)

In [ ]:
train_data['Sentiment'].value_counts()

,count
Sentiment,
pos,22761
neg,22514


In [ ]:
def normalize_arabic(text):
  # Remove diacritics
    text = re.sub(r'[\u064B-\u065F\u0670]', '', text)
  # Normalize letters variations
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    text = re.sub("ڤ", "ف", text)
    text = re.sub("چ", "ج", text)
    text = re.sub("پ", "ب", text)
    text = re.sub("ڜ", "ش", text)
    text = re.sub("ڪ", "ك", text)
    text = re.sub("ڧ", "ق", text)
    text = re.sub("ٱ", "ا", text)
    return text

In [ ]:
train_data['comment'] = train_data['comment'].apply(normalize_arabic)
test_data['comment'] = test_data['comment'].apply(normalize_arabic)

In [ ]:
train_data

,Sentiment,comment
0,pos,🚨 لزياده متابعينك ، ضيفني + لايك + رتويت + ضيف...
1,pos,تصافحنا الحياه ..🌺 لنبتسم لليوم ..لنضحك للغد ل...
2,neg,كلما #ارغمت نفسي علي عدم #تذكرك يتصارع قلبي وي...
3,neg,خساره عاد كانت نومه الظالم عباده 😔
4,neg,يخي شهالناس السعاده :(
...,...,...
45270,neg,💘💘مالتي ماكا + فوتولايز💘💘 مميزات 💑 م💊التي ماك💊...
45271,neg,حسبي الله للحين صامل 😷
45272,neg,لغه الارقام 😣
45273,neg,داخلي صرخه الم لو اطلعها ل/ صم منهاا الكون 💔 #...


In [ ]:
import emoji

def handle_emojis(text, mode='remove'):
    if mode == 'remove':
        return emoji.replace_emoji(text, '')
    elif mode == 'description':
        return emoji.demojize(text, language='ar')
    return text

In [ ]:
train_data['comment'] = train_data['comment'].apply(lambda x: handle_emojis(x, mode='description'))
test_data['comment'] = test_data['comment'].apply(lambda x: handle_emojis(x, mode='description'))

In [ ]:
def remove_arabic_noise(text):
    # Remove tatweel
    text = re.sub(r'\u0640', '', text)
    # Remove HTML tags
    text = re.sub('<.*?>', '', text)
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [ ]:
train_data['comment'] = train_data['comment'].apply(remove_arabic_noise)
test_data['comment'] = test_data['comment'].apply(remove_arabic_noise)

In [ ]:
train_data

,Sentiment,comment
0,pos,:إضاءة_سيارة_شرطة_دوارة: لزياده متابعينك ، ضيف...
1,pos,تصافحنا الحياه ..:كركديه: لنبتسم لليوم ..لنضحك...
2,neg,كلما #ارغمت نفسي علي عدم #تذكرك يتصارع قلبي وي...
3,neg,خساره عاد كانت نومه الظالم عباده :وجه_متأمل:
4,neg,يخي شهالناس السعاده :(
...,...,...
45270,neg,:قلب_وسهم::قلب_وسهم:مالتي ماكا + فوتولايز:قلب_...
45271,neg,حسبي الله للحين صامل :وجه_بكمامة:
45272,neg,لغه الارقام :وجه_جاد:
45273,neg,داخلي صرخه الم لو اطلعها ل/ صم منهاا الكون :قل...


In [ ]:
#remove puncatuations
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = r"""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""

all_punctuations = arabic_punctuations + english_punctuations

def remove_arabic_punctuations(text):
    return re.sub(f"[{re.escape(all_punctuations)}]", " ", text)

In [ ]:
train_data['comment'] = train_data['comment'].apply(remove_arabic_punctuations)
test_data['comment'] = test_data['comment'].apply(remove_arabic_punctuations)

In [ ]:
train_data

,Sentiment,comment
0,pos,إضاءة سيارة شرطة دوارة لزياده متابعينك ضيف...
1,pos,تصافحنا الحياه كركديه لنبتسم لليوم لنضحك...
2,neg,كلما ارغمت نفسي علي عدم تذكرك يتصارع قلبي وي...
3,neg,خساره عاد كانت نومه الظالم عباده وجه متأمل
4,neg,يخي شهالناس السعاده
...,...,...
45270,neg,قلب وسهم قلب وسهم مالتي ماكا فوتولايز قلب ...
45271,neg,حسبي الله للحين صامل وجه بكمامة
45272,neg,لغه الارقام وجه جاد
45273,neg,داخلي صرخه الم لو اطلعها ل صم منهاا الكون قل...


In [ ]:
def correct_arabic_text(text):
    corrections = {
        'انشاء الله': 'إن شاء الله',
        'لاكن': 'لكن',
        'انشالله': 'إن شاء الله',
        'الذي': 'الذي',
        'هاذا': 'هذا',
        'إنه': 'إنه',
    }

    for mistake, correction in corrections.items():
        text = text.replace(mistake, correction)

    return text

In [ ]:
train_data['comment'] = train_data['comment'].apply(correct_arabic_text)
test_data['comment'] = test_data['comment'].apply(correct_arabic_text)

In [ ]:
def handle_numbers_and_special_chars(text, mode='remove'):
    if mode == 'remove':
        text = re.sub(r'\d+', '', text)
        return text
    elif mode == 'normalize':
        number_map = {
                      '٠': 'صفر',
                      '١': 'واحد',
                      '٢': 'اثنان',
                      '٣': 'ثلاثة',
                      '٤': 'أربعة',
                      '٥': 'خمسة',
                      '٦': 'ستة',
                      '٧': 'سبعة',
                      '٨': 'ثمانية',
                      '٩': 'تسعة',
                      '0': 'صفر',
                      '1': 'واحد',
                      '2': 'اثنان',
                      '3': 'ثلاثة',
                      '4': 'أربعة',
                      '5': 'خمسة',
                      '6': 'ستة',
                      '7': 'سبعة',
                      '8': 'ثمانية',
                      '9': 'تسعة'
                  }
        for arabic, name in number_map.items():
            text = text.replace(arabic, name)
        return text

In [ ]:
train_data['comment'] = train_data['comment'].apply(lambda x: handle_numbers_and_special_chars(x, mode='normalize'))
test_data['comment'] = test_data['comment'].apply(lambda x: handle_numbers_and_special_chars(x, mode='normalize'))

In [ ]:
def arabizi_to_arabic(text):
    # This is a simplified conversion. A complete solution would be more complex.
    conversion_dict = {
        'a': 'ا', 'b': 'ب', 't': 'ت', 'th': 'ث', 'g': 'ج', '7': 'ح', 'kh': 'خ',
        'd': 'د', 'th': 'ذ', 'r': 'ر', 'z': 'ز', 's': 'س', 'sh': 'ش', '9': 'ص',
        '6': 'ط', '3': 'ع', 'gh': 'غ', 'f': 'ف', 'q': 'ق', 'k': 'ك', 'l': 'ل',
        'm': 'م', 'n': 'ن', 'h': 'ه', 'w': 'و', 'y': 'ي'
    }

    for latin, arabic in conversion_dict.items():
        text = text.replace(latin, arabic)

    return text

In [ ]:
train_data['comment'] = train_data['comment'].apply(arabizi_to_arabic)
test_data['comment'] = test_data['comment'].apply(arabizi_to_arabic)

In [ ]:
def normalize_elongated_words(text):
    # Remove elongation
    text = re.sub(r'(.)\1+', r'\1\1', text)
    return text

In [ ]:
train_data['comment'] = train_data['comment'].apply(normalize_elongated_words)
test_data['comment'] = test_data['comment'].apply(normalize_elongated_words)

In [ ]:
!pip -q install nltk
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
train_data['comment'] = train_data['comment'].apply(lambda x: word_tokenize(x))
test_data['comment'] = test_data['comment'].apply(lambda x: word_tokenize(x))

In [ ]:
train_data

,Sentiment,comment
0,pos,"[إضاءة, سيارة, شرطة, دوارة, لزياده, متابعينك, ..."
1,pos,"[تصافحنا, الحياه, كركديه, لنبتسم, لليوم, لنضحك..."
2,neg,"[كلما, ارغمت, نفسي, علي, عدم, تذكرك, يتصارع, ق..."
3,neg,"[خساره, عاد, كانت, نومه, الظالم, عباده, وجه, م..."
4,neg,"[يخي, شهالناس, السعاده]"
...,...,...
45270,neg,"[قلب, وسهم, قلب, وسهم, مالتي, ماكا, فوتولايز, ..."
45271,neg,"[حسبي, الله, للحين, صامل, وجه, بكمامة]"
45272,neg,"[لغه, الارقام, وجه, جاد]"
45273,neg,"[داخلي, صرخه, الم, لو, اطلعها, ل, صم, منهاا, ا..."


In [ ]:
!pip install farasapy

In [ ]:
from farasa.segmenter import FarasaSegmenter
segmenter = FarasaSegmenter()

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'farasa-api.qcri.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


100%|██████████| 241M/241M [11:23<00:00, 353kiB/s]


In [ ]:
arabic_stopwords = set(stopwords.words('arabic'))
train_data['comment'] = train_data['comment'].apply(lambda x: [word for word in x if word not in arabic_stopwords])
test_data['comment'] = test_data['comment'].apply(lambda x: [word for word in x if word not in arabic_stopwords])

In [ ]:
train_data

,Sentiment,comment
0,pos,"[إضاءة, سيارة, شرطة, دوارة, لزياده, متابعينك, ..."
1,pos,"[تصافحنا, الحياه, كركديه, لنبتسم, لليوم, لنضحك..."
2,neg,"[ارغمت, نفسي, علي, عدم, تذكرك, يتصارع, قلبي, و..."
3,neg,"[خساره, كانت, نومه, الظالم, عباده, وجه, متأمل]"
4,neg,"[يخي, شهالناس, السعاده]"
...,...,...
45270,neg,"[قلب, وسهم, قلب, وسهم, مالتي, ماكا, فوتولايز, ..."
45271,neg,"[حسبي, الله, للحين, صامل, وجه, بكمامة]"
45272,neg,"[لغه, الارقام, وجه, جاد]"
45273,neg,"[داخلي, صرخه, الم, اطلعها, صم, منهاا, الكون, ق..."


In [ ]:
from nltk.stem.isri import ISRIStemmer
stemmer = ISRIStemmer()

In [ ]:
train_data['comment'] = train_data['comment'].apply(lambda x: [stemmer.stem(word) for word in x])
test_data['comment'] = test_data['comment'].apply(lambda x: [stemmer.stem(word) for word in x])

In [ ]:
train_data

,Sentiment,comment
0,pos,"[ضءة, سير, شرط, دور, زيد, متابعينك, ضيف, ليك, ..."
1,pos,"[صفح, حيه, كركد, تسم, يوم, نضح, لغد, زرع, امل,..."
2,neg,"[رغم, نفس, علي, عدم, ذكر, صرع, قلب, نفذ, ابه, ..."
3,neg,"[خسر, كانت, نوم, ظلم, عبد, وجه, تأمل]"
4,neg,"[يخي, شهالناس, سعد]"
...,...,...
45270,neg,"[قلب, سهم, قلب, سهم, الت, اكا, فوتولايز, قلب, ..."
45271,neg,"[حسب, الل, حين, صمل, وجه, كمم]"
45272,neg,"[لغه, رقم, وجه, جاد]"
45273,neg,"[دخل, صرخ, الم, طلع, صم, نها, كون, قلب, كسر, ج..."


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
train_data['comment'] = train_data['comment'].apply(lambda x: ' '.join(x))
test_data['comment'] = test_data['comment'].apply(lambda x: ' '.join(x))

cv_tfidf = TfidfVectorizer()
X_train = cv_tfidf.fit_transform(train_data['comment'])
X_test = cv_tfidf.transform(test_data['comment'])
y_train = train_data['Sentiment']
y_test = test_data['Sentiment']

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Train Accuracy: {train_accuracy}")
print(classification_report(y_train, y_train_pred))

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))

Train Accuracy: 0.9266924351187189
              precision    recall  f1-score   support

         neg       0.94      0.91      0.93     22514
         pos       0.91      0.94      0.93     22761

    accuracy                           0.93     45275
   macro avg       0.93      0.93      0.93     45275
weighted avg       0.93      0.93      0.93     45275

Accuracy: 0.9116319444444444
              precision    recall  f1-score   support

         neg       0.93      0.89      0.91      5768
         pos       0.90      0.93      0.91      5752

    accuracy                           0.91     11520
   macro avg       0.91      0.91      0.91     11520
weighted avg       0.91      0.91      0.91     11520



In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Train Accuracy: {train_accuracy}")
print(classification_report(y_train, y_train_pred))

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))

Train Accuracy: 0.9941689674213142
              precision    recall  f1-score   support

         neg       0.99      1.00      0.99     22514
         pos       1.00      0.99      0.99     22761

    accuracy                           0.99     45275
   macro avg       0.99      0.99      0.99     45275
weighted avg       0.99      0.99      0.99     45275

Accuracy: 0.9384548611111111
              precision    recall  f1-score   support

         neg       0.95      0.93      0.94      5768
         pos       0.93      0.95      0.94      5752

    accuracy                           0.94     11520
   macro avg       0.94      0.94      0.94     11520
weighted avg       0.94      0.94      0.94     11520

